In [ ]:
from itertools import islice

from matplotlib import pyplot as plt
import matplotlib.dates as mdates
from tqdm.autonotebook import tqdm

import torch
from gluonts.evaluation import make_evaluation_predictions, Evaluator
from gluonts.dataset.repository.datasets import get_dataset
from gluonts.model.forecast import SampleForecast

from gluonts.dataset.pandas import PandasDataset
import pandas as pd
import numpy as np
from lag_llama.gluon.estimator import LagLlamaEstimator
import wandb

## Select data file to train & test: 


In [ ]:
df = pd.read_parquet("datasets/deterioration/ts_long_0.1_100_20.parquet")


In [ ]:
df.head()

In [ ]:
# Set numerical columns as float32
for col in df.columns:
    # Check if column is not of string type
    if df[col].dtype != 'object' and pd.api.types.is_string_dtype(df[col]) == False:
        df[col] = df[col].astype('float32')
        

# Create the Pandas
dataset = PandasDataset.from_long_dataframe(df, target="target", item_id="item_id")


## Set train and test set

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# Select numerical columns to normalize
numerical_columns = df.select_dtypes(include=['float32', 'float64', 'float']).columns

# Split the data first
train_size = int(len(df) * 0.8)
train_df = df.iloc[:train_size]
# Validation_df: 60-20-20
test_df = df.iloc[train_size:]



## Normalize both sets

In [ ]:
# Normalize the training data
train_df[numerical_columns] = scaler.fit_transform(train_df[numerical_columns])
# print(train_df)
# print(scaler.mean_)
# Normalize the test data using the same scaler (do not fit again)
test_df[numerical_columns] = scaler.transform(test_df[numerical_columns])

In [ ]:
# Create the Pandas datasets
dataset = {
    'train': PandasDataset.from_long_dataframe(train_df, target="target", item_id="item_id"),
    'test': PandasDataset.from_long_dataframe(test_df, target="target", item_id="item_id")
}

In [ ]:
prediction_length = 10
context_length = prediction_length*3
num_samples = 10
device=torch.device('cpu')


## Fine Tuning 
Hyper parameters tuning: 

In [ ]:
ckpt = torch.load("lag-llama.ckpt", map_location=device)
estimator_args = ckpt["hyper_parameters"]["model_kwargs"]

estimator = LagLlamaEstimator(
        ckpt_path="lag-llama.ckpt",
        prediction_length=prediction_length,
        context_length=context_length,

        # distr_output="neg_bin",
        # scaling="mean",
        nonnegative_pred_samples=True,
        aug_prob=0,
        lr=5e-4,

        # estimator args
        input_size=estimator_args["input_size"],
        n_layer=estimator_args["n_layer"],
        n_embd_per_head=estimator_args["n_embd_per_head"],
        n_head=estimator_args["n_head"],
        time_feat=estimator_args["time_feat"],

        # rope_scaling={
        #     "type": "linear",
        #     "factor": max(1.0, (context_length + prediction_length) / estimator_args["context_length"]),
        # },
        device=device,

        batch_size=64,
        num_parallel_samples=num_samples,
        trainer_kwargs = {"max_epochs": 50,}, # <- lightning trainer arguments
    )

In [ ]:
predictor = estimator.train(dataset['train'], cache_data=True, shuffle_buffer_length=1000)

In [ ]:
forecast_it, ts_it = make_evaluation_predictions(
        dataset=dataset['test'],
        predictor=predictor,
        num_samples=num_samples
    )

In [ ]:
forecasts = list(tqdm(forecast_it, total=len(dataset), desc="Forecasting batches"))

In [ ]:
print(type(forecasts))
print(len(forecasts))
print(forecasts[0])

In [ ]:
for i, forecast in enumerate(forecasts):
    samples = forecast.samples

    # Iterate over each sample and apply the inverse transform
    denormalized_samples = np.array([
        scaler.inverse_transform(sample.reshape(1, -1)).flatten()
        for sample in samples
    ])
    
    # Create a new SampleForecast object with the denormalized samples
    denormalized_forecast = SampleForecast(
        samples=denormalized_samples,
        start_date=forecast.start_date,
        item_id=forecast.item_id
    )
    forecasts[i] = denormalized_forecast

In [ ]:
tss = list(tqdm(ts_it, total=len(dataset), desc="Ground truth"))

In [ ]:
print(type(tss))
print(len(tss[0]))
print(type(tss[0]))
print(tss[0][0].shape)
print(type(tss))
print(type(tss[0]))
print(tss[0])


In [ ]:
denormalized_tss = []

for ts in tss:
    ts_copy = ts.copy()
    
    # Get the values, reshape them to fit the scaler (if necessary)
    values = ts_copy.values.reshape(-1, 1)

    # Denormalize the values using the scaler
    denormalized_values = scaler.inverse_transform(values)

    # Update the DataFrame with denormalized values
    ts_copy.iloc[:, 0] = denormalized_values.flatten()

    # Add the denormalized time series to the list
    denormalized_tss.append(ts_copy)

tss = denormalized_tss

In [ ]:
plt.figure(figsize=(20, 15))
date_formater = mdates.DateFormatter('%b, %d')
plt.rcParams.update({'font.size': 15})

# Iterate through the first 9 series, and plot the predicted samples
for idx, (forecast, ts) in islice(enumerate(zip(forecasts, tss)), 9):
    ax = plt.subplot(3, 3, idx+1)

    plt.plot(ts[-4 * prediction_length:].to_timestamp(), label="target", )
    forecast.plot( color='g')
    plt.xticks(rotation=60)
    ax.xaxis.set_major_formatter(date_formater)
    ax.set_title(forecast.item_id)

plt.gcf().tight_layout()
plt.legend()
plt.show()

In [ ]:
evaluator = Evaluator()
agg_metrics, ts_metrics = evaluator(iter(tss), iter(forecasts))

In [ ]:
agg_metrics